# Introduction to LangChain

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Initial setup

### Install dependencies

### Set API key for Groq
Click [here](https://console.groq.com/keys) to create API key for Groq, if not already created.

In [2]:
import os, json, re, getpass
from dotenv import load_dotenv

load_dotenv( override=True)

True

In [3]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key: ")

In [4]:
# if "TEST_API_KEY" not in os.environ:
#     os.environ["TEST_API_KEY"] = getpass.getpass("TEST API Key: ")

In [5]:
if os.environ["GROQ_API_KEY"]:
    print(f"Groq API Key exists and begins {os.environ["GROQ_API_KEY"][:4]}")
else:
    print("Groq API Key not set (and this is optional)")

Groq API Key exists and begins gsk_


## LangChain Components

### LLM / ChatModel

In [6]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [7]:
#Using LangChain
from langchain.chat_models import init_chat_model

model_name = "llama-3.1-8b-instant"
llm = init_chat_model(model_name, model_provider="groq") #Other Llama alternatives available are llama3-8b-8192, llama-3.3-70b-versatile

In [8]:
llm_response = llm.invoke("what is ECG?")
llm_response

AIMessage(content="ECG stands for Electrocardiogram. It is a medical test that records the electrical activity of the heart over a period of time. An ECG is a non-invasive test that uses electrodes placed on the skin to detect the electrical impulses that control the heartbeat.\n\nThe ECG machine records the electrical signals and displays them on a graph, showing the heart's rhythm and rate. This information can help healthcare professionals diagnose and monitor various heart-related conditions, such as:\n\n1. Arrhythmias (abnormal heart rhythms)\n2. Heart attacks\n3. Heart failure\n4. Coronary artery disease\n5. Cardiac hypertrophy (thickening of the heart muscle)\n6. Electrolyte imbalances\n7. Other heart conditions\n\nA typical ECG test involves placing electrodes on the chest, arms, and legs, and then recording the electrical activity of the heart for 10-30 seconds. The ECG can be performed in various settings, including:\n\n1. Doctor's offices\n2. Hospitals\n3. Emergency rooms\n4

In [9]:
print("type of response", type(llm_response))

type of response <class 'langchain_core.messages.ai.AIMessage'>


In [10]:
# print(llm_response.content)
# display(llm_response.response_metadata)
display(llm_response.usage_metadata)

{'input_tokens': 40, 'output_tokens': 332, 'total_tokens': 372}

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Detect and correct all spelling, grammatical, and punctuation errors in the provided text.\
                  Ensure proper sentence structure, clarity, and readability.\
                  Retain the core message of the original text while making the necessary corrections"),
    HumanMessage(content="hey can you send me that report by tomorrow thx"),
]

ai_response = llm.invoke(messages)
print(ai_response.content)

Here's the corrected text:

"Hi, can you send me that report by tomorrow, please?"

Changes made:
- Changed "hey" to "Hi" (proper greeting)
- Added a comma after "tomorrow" for clarity and proper punctuation
- Changed "thx" to "please" (more formal and clear expression)

This revised text is more polite and professional, and it conveys your request clearly.


In [12]:
# followup conversation
messages.append(ai_response)

In [13]:
#Ask a follow-up question
messages.append(HumanMessage(content="can you make the tone a bit informal"))

In [14]:
ai_response = llm.invoke(messages)
# print(ai_response)
print(ai_response.content)

Here's the revised text:

"Hey, can you send me that report by tomorrow?"

Changes made:
- Changed "Hi" back to "Hey" (more informal greeting)
- Removed "please" (makes the tone slightly more casual)
- Kept the comma after "tomorrow" for clarity and proper punctuation

This revised text still conveys a friendly and approachable tone while being slightly more informal.


#### Doing without LangChain

In [15]:
#Without LangChain - how would we initialize our LLM?
from openai import OpenAI

model_name = "llama-3.1-8b-instant"
llm_api = OpenAI(api_key=os.environ["GROQ_API_KEY"], base_url="https://api.groq.com/openai/v1")

In [16]:
# #Below is LangChain's messages format
# messages = [

#     SystemMessage(content="Detect and correct all spelling, grammatical, and punctuation errors in the provided text.\
#                   Ensure proper sentence structure, clarity, and readability.\
#                   Retain the core message of the original text while making the necessary corrections"),
#     HumanMessage(content="hey can you send me that report by tomorrow thx"),
# ]

#Below is messages in OpenAI format
messages_openai = [
    {'role':"system", 'content':"Detect and correct all spelling, grammatical, and punctuation errors in the provided text.\
     Ensure proper sentence structure, clarity, and readability.\
     Retain the core message of the original text while making the necessary corrections"},
     
     {'role':"user", 'content':"hey can you send me that report by tomorrow thx"}
]


In [17]:
ai_response_openai = llm_api.chat.completions.create(model= model_name,
                                messages=messages_openai)

In [18]:
ai_response_openai

ChatCompletion(id='chatcmpl-ab49cf63-a3e6-478d-a073-5b5f3df5942f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here\'s a corrected version of the text:\n\n"Hi, could you please send me that report by tomorrow? Thank you."\n\nI made a few changes to improve the clarity and grammar of the message:\n\n- Started with a more formal greeting ("Hi" instead of "hey")\n- Added a more appropriate verb ("could you please send" instead of the informal "can you send")\n- Changed the phrase to "by tomorrow" to convey a more specific deadline\n- Added a closing ("Thank you") to make the message more polite', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1757490449, model='llama-3.1-8b-instant', object='chat.completion', service_tier='on_demand', system_fingerprint='fp_46fc01befd', usage=CompletionUsage(completion_tokens=107, prompt_tokens=89, total_tokens=196, completion_tokens_det

In [19]:
ai_response_openai_formatted = ai_response_openai.choices[0].message.content
print(ai_response_openai_formatted)

Here's a corrected version of the text:

"Hi, could you please send me that report by tomorrow? Thank you."

I made a few changes to improve the clarity and grammar of the message:

- Started with a more formal greeting ("Hi" instead of "hey")
- Added a more appropriate verb ("could you please send" instead of the informal "can you send")
- Changed the phrase to "by tomorrow" to convey a more specific deadline
- Added a closing ("Thank you") to make the message more polite


In [20]:
messages_openai

[{'role': 'system',
  'content': 'Detect and correct all spelling, grammatical, and punctuation errors in the provided text.     Ensure proper sentence structure, clarity, and readability.     Retain the core message of the original text while making the necessary corrections'},
 {'role': 'user',
  'content': 'hey can you send me that report by tomorrow thx'}]

In [21]:
#Append the AI message
messages_openai.append(
    {'role': "assistant",
     'content': ai_response_openai_formatted}
)

In [22]:
messages_openai

[{'role': 'system',
  'content': 'Detect and correct all spelling, grammatical, and punctuation errors in the provided text.     Ensure proper sentence structure, clarity, and readability.     Retain the core message of the original text while making the necessary corrections'},
 {'role': 'user',
  'content': 'hey can you send me that report by tomorrow thx'},
 {'role': 'assistant',
  'content': 'Here\'s a corrected version of the text:\n\n"Hi, could you please send me that report by tomorrow? Thank you."\n\nI made a few changes to improve the clarity and grammar of the message:\n\n- Started with a more formal greeting ("Hi" instead of "hey")\n- Added a more appropriate verb ("could you please send" instead of the informal "can you send")\n- Changed the phrase to "by tomorrow" to convey a more specific deadline\n- Added a closing ("Thank you") to make the message more polite'}]

In [23]:
#Ask a follow-up question
#LangChain version below
# messages.append(HumanMessage(content="can you make the tone a bit informal"))

#OpenAI version below
messages_openai.append(
    {'role':"user",
     'content':"can you make the tone a bit informal"}
)

In [24]:
messages_openai

[{'role': 'system',
  'content': 'Detect and correct all spelling, grammatical, and punctuation errors in the provided text.     Ensure proper sentence structure, clarity, and readability.     Retain the core message of the original text while making the necessary corrections'},
 {'role': 'user',
  'content': 'hey can you send me that report by tomorrow thx'},
 {'role': 'assistant',
  'content': 'Here\'s a corrected version of the text:\n\n"Hi, could you please send me that report by tomorrow? Thank you."\n\nI made a few changes to improve the clarity and grammar of the message:\n\n- Started with a more formal greeting ("Hi" instead of "hey")\n- Added a more appropriate verb ("could you please send" instead of the informal "can you send")\n- Changed the phrase to "by tomorrow" to convey a more specific deadline\n- Added a closing ("Thank you") to make the message more polite'},
 {'role': 'user', 'content': 'can you make the tone a bit informal'}]

In [25]:
ai_response_openai = llm_api.chat.completions.create(
    model=model_name,
    messages=messages_openai
)

In [26]:
type(ai_response_openai)

openai.types.chat.chat_completion.ChatCompletion

In [27]:
print(ai_response_openai.choices[0].message.content)

Here's a slightly more informal version of the text:

"Hey, can I get that report by tomorrow?"

I made some minor changes to make the tone more casual:

- Returned to the more informal greeting ("Hey" instead of "Hi")
- Shortened the sentence to make it more direct
- Removed the "Thank you" to keep the tone casual


In [28]:
#Explain concept of context length here - Context length = input tokens + completion tokens 

### Output Parsers

In [29]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

parser = StrOutputParser()

*StrOutputParser* is a runnable object.

In [30]:
result = llm.invoke(messages)

parser.invoke(result)

'Here\'s the revised text:\n\n"Hey, can you send me that report by tomorrow? Thanks!"\n\nChanges made:\n- Changed "Hi" back to "Hey" (more informal greeting)\n- Removed the comma after "tomorrow" (less formal and more conversational)\n- Changed "please" to "Thanks!" (more casual and friendly expression)\n\nThis revised text has a friendly, yet still professional tone.'

In [31]:
result

AIMessage(content='Here\'s the revised text:\n\n"Hey, can you send me that report by tomorrow? Thanks!"\n\nChanges made:\n- Changed "Hi" back to "Hey" (more informal greeting)\n- Removed the comma after "tomorrow" (less formal and more conversational)\n- Changed "please" to "Thanks!" (more casual and friendly expression)\n\nThis revised text has a friendly, yet still professional tone.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 192, 'total_tokens': 276, 'completion_time': 0.111757862, 'prompt_time': 0.529207673, 'queue_time': 0.047274416, 'total_time': 0.640965535}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_46fc01befd', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ebedec27-c41c-4dca-8b11-32627688e1ec-0', usage_metadata={'input_tokens': 192, 'output_tokens': 84, 'total_tokens': 276})

In [40]:
messages = [
    SystemMessage(content="""You are an expert in writing analysis. You will receive a message from a user, and your job is to evaluate the text based on the following attributes:
1. clarity: Is the message clear, or unclear?
2. grammar_quality: Are there any grammatical issues? Possible values: correct, minor issues, major issues.
3. tone: Analyze whether the tone is neutral, formal, or informal.
4. suggestions: Offer brief improvement suggestions for clarity, grammar, or tone.

Return a structured JSON object with these four attributes."""),
    HumanMessage(content="Hey, could you please send me that report by tomorrow? Thank you.")
]
response = llm.invoke(messages)

In [41]:
response

AIMessage(content='Here is the analysis of the message in a JSON format:\n\n```json\n{\n  "clarity": 8,\n  "grammar_quality": "correct",\n  "tone": "informal",\n  "suggestions": [\n    "Consider adding more context or details about the report to ensure clarity.",\n    "To make the tone even more friendly and professional, you could add a smiley face or a polite phrase like \'I appreciate your help\'.",\n    "If the report is for a formal or official purpose, consider using a more formal tone."\n  ]\n}\n```\n\nHowever, since it seems you asked for a report which we don\'t have in this context, I can make an assumption that you\'re asking for an analysis of a report you\'re about to send. If that\'s the case, please provide the report and I\'ll give you a more accurate analysis.\n\nAssuming that\'s the case I will wait for the report.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 191, 'prompt_tokens': 159, 'total_tokens': 350, 'completion_time': 0.938880

In [42]:
response.content

'Here is the analysis of the message in a JSON format:\n\n```json\n{\n  "clarity": 8,\n  "grammar_quality": "correct",\n  "tone": "informal",\n  "suggestions": [\n    "Consider adding more context or details about the report to ensure clarity.",\n    "To make the tone even more friendly and professional, you could add a smiley face or a polite phrase like \'I appreciate your help\'.",\n    "If the report is for a formal or official purpose, consider using a more formal tone."\n  ]\n}\n```\n\nHowever, since it seems you asked for a report which we don\'t have in this context, I can make an assumption that you\'re asking for an analysis of a report you\'re about to send. If that\'s the case, please provide the report and I\'ll give you a more accurate analysis.\n\nAssuming that\'s the case I will wait for the report.'

In [43]:
json_response = JsonOutputParser().invoke(response.content)
json_response

{'clarity': 8,
 'grammar_quality': 'correct',
 'tone': 'informal',
 'suggestions': ['Consider adding more context or details about the report to ensure clarity.',
  "To make the tone even more friendly and professional, you could add a smiley face or a polite phrase like 'I appreciate your help'.",
  'If the report is for a formal or official purpose, consider using a more formal tone.']}

In [44]:
var1 = 'json{"clarity": "unclear"}'
print(var1)

json{"clarity": "unclear"}


In [45]:
var1

'json{"clarity": "unclear"}'

In [46]:
# JsonOutputParser().invoke(var1)

In [47]:
type(json_response)

dict

In [48]:
type(var1)

str

In [49]:
print(response.content)

Here is the analysis of the message in a JSON format:

```json
{
  "clarity": 8,
  "grammar_quality": "correct",
  "tone": "informal",
  "suggestions": [
    "Consider adding more context or details about the report to ensure clarity.",
    "To make the tone even more friendly and professional, you could add a smiley face or a polite phrase like 'I appreciate your help'.",
    "If the report is for a formal or official purpose, consider using a more formal tone."
  ]
}
```

However, since it seems you asked for a report which we don't have in this context, I can make an assumption that you're asking for an analysis of a report you're about to send. If that's the case, please provide the report and I'll give you a more accurate analysis.

Assuming that's the case I will wait for the report.


In [50]:
json_response['clarity']

8

### Chain (LCEL)

In [51]:
chain = llm | JsonOutputParser()

chain.invoke(messages)

{'clarity': 'unclear',
 'grammar_quality': 'minor issues',
 'tone': 'informal',
 'suggestions': ['Consider adding more context about the report, such as what kind of report it is and what information it should contain.',
  "You could specify a more exact deadline, instead of just 'tomorrow', to avoid any confusion.",
  "If you're expecting a formal response, consider using a more formal tone in your request."]}

### PromptTemplate

In [52]:
# my_str = "Hello world, this is the tone - {tone}"

In [53]:
# my_str.format(tone = "Happy")

In [54]:
# tone = "Happy"
# my_str = f"Hello world, this is the tone - {tone}"
# print(my_str)

In [55]:
my_str = """Detect and correct all spelling, grammatical, and punctuation errors in the provided text.
Ensure proper sentence structure, clarity, and readability.

Tone Adjustment: {tone}

Communication Style: {communication_style}

Retain the core message of the original text while making the necessary corrections and tone adjustments.
Suggest appropriate phrasing and formatting based on the tone and communication style."""

In [56]:
print(my_str.format(tone="Happy", communication_style = "Formal"))

Detect and correct all spelling, grammatical, and punctuation errors in the provided text.
Ensure proper sentence structure, clarity, and readability.

Tone Adjustment: Happy

Communication Style: Formal

Retain the core message of the original text while making the necessary corrections and tone adjustments.
Suggest appropriate phrasing and formatting based on the tone and communication style.


In [57]:
from langchain_core.prompts import ChatPromptTemplate
system_message_template = """Detect and correct all spelling, grammatical, and punctuation errors in the provided text.
Ensure proper sentence structure, clarity, and readability.

Tone Adjustment: {tone}

Communication Style: {communication_style}

Retain the core message of the original text while making the necessary corrections and tone adjustments.
Suggest appropriate phrasing and formatting based on the tone and communication style."""
template = ChatPromptTemplate([
    ("system", system_message_template),
    ("human", "{user_input}"),
])

In [58]:
template.input_variables

['communication_style', 'tone', 'user_input']

In [59]:
#How does this work without LCEL = LangChain Expression LangChain

In [60]:
tone = 'Rewrite the message in a professional, polite, and structured manner. \
Suitable for business emails, official reports, or any context requiring formality and respect.'

communication_style = 'Messages should be clear, structured, and formal or neutral depending on the context. \
Introductions, conclusions, and appropriate sign-offs should be added if missing.'

user_input = """Can u send me the data by eod pls?"""

In [61]:
#First step
formatted_template = template.invoke({"communication_style": communication_style,
    "tone":tone,
    "user_input":user_input
})
formatted_template

ChatPromptValue(messages=[SystemMessage(content='Detect and correct all spelling, grammatical, and punctuation errors in the provided text.\nEnsure proper sentence structure, clarity, and readability.\n\nTone Adjustment: Rewrite the message in a professional, polite, and structured manner. Suitable for business emails, official reports, or any context requiring formality and respect.\n\nCommunication Style: Messages should be clear, structured, and formal or neutral depending on the context. Introductions, conclusions, and appropriate sign-offs should be added if missing.\n\nRetain the core message of the original text while making the necessary corrections and tone adjustments.\nSuggest appropriate phrasing and formatting based on the tone and communication style.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can u send me the data by eod pls?', additional_kwargs={}, response_metadata={})])

In [62]:
type(formatted_template)

langchain_core.prompt_values.ChatPromptValue

In [63]:
#Second step
response = llm.invoke(formatted_template)

In [64]:
type(response)

langchain_core.messages.ai.AIMessage

In [65]:
response.content

"Here's a rewritten version of the message in a professional and polite tone:\n\nDear [Recipient's Name],\n\nI hope this message finds you well. I am writing to request the data by the end of the day today. Could you please provide me with the required data as soon as possible so that I can proceed with the next steps?\n\nThank you for your prompt attention to this matter. I appreciate your cooperation and look forward to receiving the data.\n\nBest regards,\n[Your Name]\n\nAlternatively, if you want a more concise version:\n\nDear [Recipient's Name],\n\nCould you please provide the required data by the end of the day today? I appreciate your prompt attention to this matter and look forward to receiving the data.\n\nThank you for your cooperation.\n\nBest regards,\n[Your Name]\n\nIf you need to include a deadline or specific time, you can add that to the message:\n\nDear [Recipient's Name],\n\nCould you please provide the required data by 5:00 PM today? I appreciate your prompt attenti

In [66]:
#Third step
final_parsed_result = StrOutputParser().invoke(response)
final_parsed_result

"Here's a rewritten version of the message in a professional and polite tone:\n\nDear [Recipient's Name],\n\nI hope this message finds you well. I am writing to request the data by the end of the day today. Could you please provide me with the required data as soon as possible so that I can proceed with the next steps?\n\nThank you for your prompt attention to this matter. I appreciate your cooperation and look forward to receiving the data.\n\nBest regards,\n[Your Name]\n\nAlternatively, if you want a more concise version:\n\nDear [Recipient's Name],\n\nCould you please provide the required data by the end of the day today? I appreciate your prompt attention to this matter and look forward to receiving the data.\n\nThank you for your cooperation.\n\nBest regards,\n[Your Name]\n\nIf you need to include a deadline or specific time, you can add that to the message:\n\nDear [Recipient's Name],\n\nCould you please provide the required data by 5:00 PM today? I appreciate your prompt attenti

In [67]:
# #With LCEL
# proof_read_chain = template | llm ##RunnableSequence
# final_response = proof_read_chain.invoke(
#     {"communication_style": communication_style,
#     "tone":tone,
#     "user_input":user_input
# })
# final_response
# type(final_response)

In [68]:
#With LCEL
proof_read_chain = template | llm | StrOutputParser()
proof_read_chain

ChatPromptTemplate(input_variables=['communication_style', 'tone', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['communication_style', 'tone'], input_types={}, partial_variables={}, template='Detect and correct all spelling, grammatical, and punctuation errors in the provided text.\nEnsure proper sentence structure, clarity, and readability.\n\nTone Adjustment: {tone}\n\nCommunication Style: {communication_style}\n\nRetain the core message of the original text while making the necessary corrections and tone adjustments.\nSuggest appropriate phrasing and formatting based on the tone and communication style.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x117069f40>, async_client=<groq.

In [69]:
type(proof_read_chain)

langchain_core.runnables.base.RunnableSequence

In [70]:
final_response = proof_read_chain.invoke(
    {"communication_style": communication_style,
    "tone":tone,
    "user_input":user_input
})
final_response

"Here's a rewritten version of the message in a professional, polite, and structured manner:\n\nDear [Recipient's Name],\n\nI am writing to request that you provide me with the requested data by the end of the day today. This will enable me to meet my deadlines and progress with the project accordingly.\n\nThank you for your prompt attention to this matter. If there are any issues or concerns that may affect the timely delivery of the data, please do not hesitate to inform me.\n\nI appreciate your cooperation and look forward to receiving the data by the end of the day.\n\nBest regards,\n[Your Name]\n\nAlternatively, you could also use a more concise version:\n\nDear [Recipient's Name],\n\nCould you please provide the requested data by the end of the day today? This will help me meet my deadlines and move forward with the project.\n\nThank you for your prompt attention to this matter.\n\nBest regards,\n[Your Name]"

In [71]:
# proof_read_chain.input_schema.model_json_schema()

In [72]:
tone_map = {
    "Formal": "Rewrite the message in a professional, polite, and structured manner. Suitable for business emails, official reports, or any context requiring formality and respect.",
    "Informal": "Rewrite in a casual, friendly, and conversational style. Appropriate for personal communications, friendly chats, or informal emails.",
    "Neutral": "Rewrite in a balanced tone that is neither overly formal nor too casual. Suitable for most general communications where a middle-ground tone is required."
}
communication_style_map = {
    "Email": "Messages should be clear, structured, and formal or neutral depending on the context. Introductions, conclusions, and appropriate sign-offs should be added if missing.",
    "General": "This covers most forms of communication and will aim for clarity and coherence. The tone can vary as per the user's choice.",
    "Instant Messaging": "Focus on brevity, clarity, and informality, using conversational phrasing suitable for quick back-and-forth exchanges.",
    "Business Instant Messaging": "Maintain a professional but conversational tone. Messages should be concise and efficient, avoiding unnecessary formalities but keeping the language respectful."
}

In [73]:
tone = "Formal" # Formal, Informal, Neutral
communication_style = "Email" # Email, General, Instant Messaging, Business Instant Messaging
user_input = """Can u send me the data by eod pls?"""

chain_output = proof_read_chain.invoke(dict(
    tone=tone_map[tone],
    communication_style = communication_style_map[communication_style],
    user_input = user_input
))
print(chain_output)

Here's a rewritten version of the message in a professional, polite, and structured manner:

Dear [Recipient],

I would greatly appreciate it if you could provide me with the requested data by the end of the day today. This will enable me to complete my tasks efficiently and meet the necessary deadlines.

Thank you for your prompt attention to this matter, and I look forward to receiving the data at your earliest convenience.

Best regards,
[Your Name]

Alternatively, if you want to make it even more concise:

Dear [Recipient],

Could you please provide me with the requested data by the end of the day today? This will help me complete my tasks on schedule.

Thank you for your assistance.

Best regards,
[Your Name]

In either case, I've added a polite introduction, a clear request for the data, and a polite expression of gratitude. I've also included a professional sign-off and your name.
